# <font color=red>Innsending av mva-melding til altinn3</font>

Test-script for å laste opp en mva-melding til altinn3-appen på tt02 miljøet i Altinn Studio.

## <font color=red>Sjekke at du har installert de nødvendige bibliotekene.</font>

In [ ]:
try:
    import requests
    import base64
    import xmltodict
    import xml.dom.minidom
    from Steg import hent_xml_filer, hente_altinntoken, hente_instans, last_opp_konvolutt, last_opp_melding
    from Steg import last_opp_vedlegg, logge_inn_idporten, oppdatere_prosess, opprett_instans, slette_vedlegg
    from InnsendingServices import Miljo, InnloggingType, printing, App
    from portalmelding import valider_melding
except ImportError as e:
    try: 
        print("Mangler en avhengighet, installer dem via pip")
        !pip install python-jose[cryptography] cryptography
        !pip install xmltodict
        !pip install requests
        import xmltodict
        import requests
        from Steg import hent_xml_filer, hente_altinntoken, hente_instans, last_opp_konvolutt, last_opp_melding
        from Steg import last_opp_vedlegg, logge_inn_idporten, oppdatere_prosess, opprett_instans, slette_vedlegg
        from InnsendingServices import Miljo, InnloggingType, printing, App
        from portalmelding import valider_melding
    except ImportError as err:
        print("Mangler en avhengighet, installer dem via pip")
        !python3 -m pip install python-jose[cryptography] cryptography
        !python3 -m pip install xmltodict
        !python3 -m pip install requests
        import xmltodict
        import requests
        from Steg import hent_xml_filer, hente_altinntoken, hente_instans, last_opp_konvolutt, last_opp_melding
        from Steg import last_opp_vedlegg, logge_inn_idporten, oppdatere_prosess, opprett_instans, slette_vedlegg
        from InnsendingServices import Miljo, InnloggingType, printing, App
        from portalmelding import valider_melding

## <font color=red>Definere variabler</font>

In [ ]:
slette_forrige_vedlegg = False  # Hvis man vil slette et vedlegg man har sendt inn ved feil.

# Trenger ikke å gjøre noe med.
# Bare for intern testing
miljo = Miljo.tt02  # tt02, local
innloggings_metode = InnloggingType.idporten  # idporten, maskinporten
domene = miljo.value  # Domenet til appen på Altinn Studio
app = App.ETM2.value  # Hvilken app repository som kjøres

header = None

# Husk å legg inn org_nummer du ønsker å sende inn for.
# Merk at personen som skal sende inn må ha nødvendig roller/rettigheter i altinn studio appen,
# for å få lov til å sende inn.
org_nummer = "123456789"


## <font color=red>Definere MvaMelding og MvaMeldingInnsending.</font>

In [ ]:
melding_xml, konvolutt_xml = hent_xml_filer.hent()

## <font color=red>Generer ID-porten token</font>
Tokenet er gyldig i 300 sekunder, rekjør denne biten om du ikke har kommet frem til altinntoken innen den tid.

In [ ]:
if innloggings_metode == InnloggingType.idporten:
    header = logge_inn_idporten.hent_idtoken()

## <font color=red>Valider meldingen(Valgfritt)</font>

In [ ]:
if innloggings_metode == InnloggingType.idporten:
  validering = valider_melding(dict(header), xml=melding_xml)

## <font color=red>Hente AltinnToken</font>

In [ ]:
if innloggings_metode != InnloggingType.maskinporten:
  altinn_token = hente_altinntoken.hent(innloggings_metode.name, header)

## <font color=red>Opprett ny instans</font>

In [ ]:
instans = opprett_instans.oprett(domene, altinn_token, app, org_nummer)

partyId = instans['instanceOwner']['partyId']
instanceGuid = instans['data'][0]['instanceGuid']
dataId = instans['data'][0]['id']

instansUrl = domene + instans['appId'] + "/instances/" + partyId + "/" + instanceGuid
mvaMeldingInnsendingUrl  = instansUrl + "/data/" + dataId
mvaMeldingUrl = instansUrl + "/data?dataType=mvamelding"
mvaMeldingVedleggUrl  = instansUrl + "/data?dataType=vedlegg"

## <font color=red>Last opp MvaMeldingInnsending</font>

In [ ]:
last_opp_konvolutt.last_opp(miljo.name, mvaMeldingInnsendingUrl, konvolutt_xml, altinn_token)

## <font color=red>Last opp MvaMelding</font>

In [ ]:
last_opp_melding.last_opp(miljo.name, mvaMeldingUrl, melding_xml, altinn_token)

## <font color=red>Last opp Vedlegg(valgfritt)</font>

#### .xml vedlegg

In [ ]:
xml_vedlegg = last_opp_vedlegg.last_opp(miljo.name, mvaMeldingVedleggUrl, "mva-vedlegg.xml", "text/xml" , altinn_token)

#### .pdf vedlegg

In [ ]:
pdf_vedlegg = last_opp_vedlegg.last_opp(miljo.name, mvaMeldingVedleggUrl, "pdf-vedlegg.pdf", "application/pdf", altinn_token)

#### .png vedlegg

In [ ]:
png_vedlegg = last_opp_vedlegg.last_opp(miljo.name, mvaMeldingVedleggUrl, "png-vedlegg.png", "image/png", altinn_token)

## <font color=red>Send Inn MvaMeldingInnsending</font>

In [ ]:
avslutteProsess = oppdatere_prosess.neste(miljo.name, instansUrl, altinn_token)

## <font color=red>Hente Instansen</font>

In [ ]:
instans = hente_instans.hent(miljo.name, instansUrl, altinn_token)